In [64]:
#import required packages
import numpy as np
import random
import csv
from sklearn.decomposition import TruncatedSVD
import string
from collections import Counter
import scipy.stats
from itertools import product

In [2]:
# function for R-squared
def r_sq(x, y):
    cor = scipy.stats.pearsonr(x, y)
    return cor[0]**2

In [3]:
def sel_second(x):
    return x[1]

In [4]:
# function to normalize all vectors in a matrix
def normalize_matrix(mat):
    sq_mat = mat**2
    sum_sq = np.sum(sq_mat, axis=1)
    mag = np.sqrt(sum_sq)
    mag[mag == 0] = 1
    normed_mat = np.transpose((np.transpose(mat) / mag))
    return normed_mat

Make Orthography vectors

In [10]:
#load tasa word list and nonwords
with open('tasa_word_list.txt', 'r') as f:
    df = f.readlines()
all_words = [i.strip() for i in df]

    SERIOL2

In [11]:
#get bigrams and weights for seriol2
def extract_bigrams(word, weights=[1, 0.7, 0.5]):
    seriol_word = '*' + word + '*'
    bigrams = []
    for i in range(0, len(seriol_word)):
        for x in range(1, 4):
            if (i+x) < len(seriol_word):
                bigrams.append((seriol_word[i] + seriol_word[i+x], weights[x-1]))
    bigrams = [i for i in bigrams if i[0]!='**']
    return bigrams

In [12]:
# find all unique bigrams
unique_strings = []
for i in all_words:
    unique_strings.extend(list(set([x[0] for x in extract_bigrams(i)])))
unique_strings = list(set(unique_strings))

In [13]:
# create blank matrix
ortho_matrix = np.zeros((len(all_words), len(unique_strings)))

In [14]:
# create dictionary for all words
ortho_dic = {}
for i in range(0, len(all_words)):
    ortho_dic[all_words[i]] = i

In [15]:
# create dictionary for all bigrams
string_dic = {}
for i in range(0, len(unique_strings)):
    string_dic[unique_strings[i]] = i

In [16]:
# create word-by-bigram matrix
for i in all_words:
    for x in extract_bigrams(i):
        ortho_matrix[ortho_dic[i]][string_dic[x[0]]] += x[1]

In [17]:
# set up for singular value decomposition (SVD) (300 dimensions)
svd = TruncatedSVD(n_components=300, algorithm='arpack')

In [18]:
# perform SVD (takes a little time)
ortho_matrix = svd.fit_transform(ortho_matrix)

In [19]:
# normalize matrix
seriol_matrix = normalize_matrix(ortho_matrix)

In [20]:
# function for finding orthographic neighbors (not required to simulate memory)
def find_neighbors(x, dic, mat, n=30):
    simvec = mat[dic[x]] @ np.transpose(mat)
    sim_list = list(zip([i for i in dic], simvec))
    sim_list.sort(key=sel_second, reverse=True)
    return sim_list[1:(n+1)]

In [45]:
# example 
find_neighbors('twice', ortho_dic, seriol_matrix)

[('tice', 0.8526681804155765),
 ('stice', 0.7199674626781277),
 ('ice', 0.6876699564332858),
 ('titmice', 0.6762183010593561),
 ('wince', 0.670086823466045),
 ('twite', 0.6599433981335214),
 ('twine', 0.6591313007351359),
 ('tic', 0.6570869048941395),
 ('wic', 0.6299350675702022),
 ('wickie', 0.6247972531746894),
 ('weicek', 0.62109491426506),
 ('notice', 0.6147037932144468),
 ('entice', 0.6141692284298366),
 ('tie', 0.6056609354283639),
 ('vice', 0.6054667344390299),
 ('triced', 0.6040850903750908),
 ('tricep', 0.6029067954315054),
 ('twitched', 0.6016352809941627),
 ('hice', 0.5989964481380832),
 ('thrice', 0.5987590930625922),
 ('pice', 0.5972353277017325),
 ('dice', 0.595743974145482),
 ('thicke', 0.595338083728224),
 ('mice', 0.5933888051856302),
 ('icie', 0.5917210271172807),
 ('lice', 0.5893059746780324),
 ('wickwire', 0.5877997658232164),
 ('twinkie', 0.5869434704395182),
 ('twitches', 0.5862649688015245),
 ('nice', 0.5862296321932584)]

    OrBEAGLE

In [33]:
# Get letter strings according to terminal-relative encoding (Cox et al., 2011)
def get_letter_strings(word):
    singles = [i for i in word]
    bigrams = list(zip(word[:-1], word[1:]))
    bigrams = ["".join(i) for i in bigrams]
    t1 = []
    if len(word)>2:
        t1.append(bigrams[0])
        t1.extend(tuple(product(*[[singles[0] + '_'], singles[2:]])))
        t1.extend(tuple(product(*[singles[:-2], ['_' + singles[-1]]]))[1:])
        t1.append(bigrams[-1])
        t1.append((singles[0], bigrams[1]))
        t1.extend(tuple(product(*[[singles[0] + '_'], bigrams[2:]])))
        t1.extend(tuple(product(*[bigrams[:-2], ['_' + singles[-1]]])))
        t1.append((bigrams[-2], singles[-1]))
        t1 = ["".join(i) for i in t1]
    return singles + bigrams + t1

In [34]:
# make an object with all letters and a space symbol
alphabet = string.ascii_lowercase + '_'

In [35]:
# make a random vector for each letter (and space symbol)
letter_dic = {}
for i in alphabet:
    letter_dic[i] = np.random.normal(0, 1/np.sqrt(1024), 1024)

In [36]:
# function for circular convolution
def conv_circ(signal, ker):
    return np.real(np.fft.ifft(np.fft.fft(signal)*np.fft.fft(ker)))

In [37]:
# make arrays to randomly permute letters to the left and right
num = [i for i in range(0, 1024)]
E1 = np.random.permutation(num)
E2 = np.random.permutation(num)

In [38]:
# make a vector representation for each substring
def make_string_vec(string):
    s_vec = letter_dic[string[0]].copy()
    if len(string) > 1:
        for i in string[1:]:
            s_vec = conv_circ(s_vec[E1], letter_dic[i][E2])
    return s_vec

In [39]:
# make dictionary for all words
ortho_dic = {}
for i in range(0, len(all_words)):
    ortho_dic[all_words[i]] = i

In [40]:
# make vector representations for all words (make vectors for substrings and add these up) (takes a while)
ortho_matrix = []
for i in all_words:
    word_vec = np.zeros(1024)
    sub = get_letter_strings(i)
    for x in sub:
        word_vec += make_string_vec(x)
    ortho_matrix.append(word_vec)

In [41]:
# convert list to numpy matrix
ortho_matrix = np.array(ortho_matrix)

In [42]:
# normalize matrix
orbeagle_matrix = normalize_matrix(ortho_matrix)

In [43]:
# function to find orthographic neighbors
def find_neighbors(x, dic, mat, n=30):
    simvec = mat[dic[x]] @ np.transpose(mat)
    sim_list = list(zip([i for i in dic], simvec))
    sim_list.sort(key=sel_second, reverse=True)
    return sim_list[1:(n+1)]

In [46]:
#example
find_neighbors('twice', ortho_dic, orbeagle_matrix)

[('twine', 0.6706159011044597),
 ('twite', 0.6444096467805196),
 ('tice', 0.6106410993325848),
 ('twinge', 0.6051627973331898),
 ('titmice', 0.5985045922500533),
 ('twinkie', 0.5873305815328165),
 ('thrice', 0.5786932973046923),
 ('twopence', 0.5719920921611105),
 ('wince', 0.5588220857487379),
 ('ice', 0.5525291947522528),
 ('twinkle', 0.54649902619543),
 ('twiglike', 0.5426615341497039),
 ('artifice', 0.494531575404631),
 ('thwickle', 0.48841685397636825),
 ('stice', 0.4793642547357937),
 ('twiddle', 0.4712660267467289),
 ('lattice', 0.4674149489816697),
 ('twig', 0.46524137120745435),
 ('tw', 0.4649184292348183),
 ('juice', 0.46326267837925417),
 ('timepiece', 0.4628480911544469),
 ('injustice', 0.46132469431096024),
 ('intertwine', 0.4605508056378591),
 ('vice', 0.4585236171298746),
 ('hotwire', 0.4559811161261165),
 ('dice', 0.44945319063383093),
 ('twines', 0.44938711421228766),
 ('withe', 0.4485482277971237),
 ('armistice', 0.44756911967791757),
 ('witte', 0.44489765081652954)]

Directed Forgetting Simulations

    Load experiment 1 stimuli lists

In [47]:
test_a = []
with open('exp1testlistA.csv', 'r') as f:
    csvreader = csv.reader(f)
    for i in csvreader:
        test_a.append(i)

In [48]:
test_b = []
with open('exp1testlistB.csv', 'r') as f:
    csvreader = csv.reader(f)
    for i in csvreader:
        test_b.append(i)

In [49]:
test_c = []
with open('exp1testlistC.csv', 'r') as f:
    csvreader = csv.reader(f)
    for i in csvreader:
        test_c.append(i)

In [50]:
test_d = []
with open('exp1testlistD.csv', 'r') as f:
    csvreader = csv.reader(f)
    for i in csvreader:
        test_d.append(i)

    Load experiment 2 stimuli lists

In [51]:
test_a_2 = []
with open('exp2testlistA.csv', 'r') as f:
    csvreader = csv.reader(f)
    for i in csvreader:
        test_a_2.append(i)

In [52]:
test_b_2 = []
with open('exp2testlistB.csv', 'r') as f:
    csvreader = csv.reader(f)
    for i in csvreader:
        test_b_2.append(i)

In [53]:
test_c_2 = []
with open('exp2testlistC.csv', 'r') as f:
    csvreader = csv.reader(f)
    for i in csvreader:
        test_c_2.append(i)

In [54]:
test_d_2 = []
with open('exp2testlistD.csv', 'r') as f:
    csvreader = csv.reader(f)
    for i in csvreader:
        test_d_2.append(i)

In [55]:
# function to set up stimuli list for running simulation
def get_stim_list(df):
    clean_df = df[1:]
    r_old = [i[1].lower() for i in clean_df if i[2]=='R' and i[4]=='old']
    f_old = [i[1].lower() for i in clean_df if i[2]=='F' and i[4]=='old']
    r_new = [i[1].lower() for i in clean_df if i[2]=='R' and i[4]=='new']
    f_new = [i[1].lower() for i in clean_df if i[2]=='F' and i[4]=='new']
    new_new = [i[1].lower() for i in clean_df if i[2]=='New' and i[4]=='new']
    return r_old + f_old + r_new + f_new + new_new

In [56]:
# make matrix for a subset of items (e.g., items to be encoded or items on recognition test)
def make_matrix(word_list, ortho_matrix):
    mem = []
    for i in word_list:
        mem.append(ortho_matrix[ortho_dic[i]])
    return np.array(mem)

In [57]:
# function for retrieval
def echo_intensity(probes, memory, tau=3):
    normed_memory = normalize_matrix(memory)
    similarities = probes @ np.transpose(normed_memory)
    if tau == 2:
        activations = similarities*(abs(similarities))
    elif tau == 4:
        activations = similarities*(abs(similarities))*similarities*(abs(similarities))
    elif tau == 6:
        activations = similarities*(abs(similarities))*similarities*(abs(similarities))*similarities*(abs(similarities))
    else:
        activations = similarities**tau
    activations = np.sum(activations, axis=1)
    return activations

In [71]:
# Run Experiment 1 simulation with OrBEAGLE representations
emp_data = np.array([0.72619048, 0.52947846, 0.61111111, 0.42630385, 0.16326531]) # empirical data for computing r-squared
ls = [.01, .03, .05, .1] # learning rate
fs = [2/3, 4/5] # directed forgetting parameter
t = 3 # tau parameter
p_old = 50 # percent of items to be classifed "OLD"
print(['L', 'F', 'mean/sd', ['R old', 'F old', 'R rel', 'F rel', 'Unrel'], 'R-squared'])
for F in fs:
    for l in ls:
        f = l * F
        sim_list = []
        for s in range(0, 800):
            if s <= 200:
                stims = get_stim_list(test_a)
            elif s <= 400:
                stims = get_stim_list(test_b)
            elif s <= 600:
                stims = get_stim_list(test_c)
            else:
                stims = get_stim_list(test_d)
            memory = make_matrix(stims[0:72], orbeagle_matrix) # get vectors for encoded items
            memory[0:36] = memory[0:36] * np.random.choice([0, 1], size=(36, len(memory[0])), p=[1-l, l]) # randomly encode elements with probability L
            memory[36:72] = memory[36:72] * np.random.choice([0, 1], size=(36, len(memory[0])), p=[1-f, f]) # randomly encode elements with probability L*F
            probes = make_matrix(stims, orbeagle_matrix) # get vectors for recognition probes
            familiarities = echo_intensity(probes, memory, tau=t) # retrival: compute familairity for all probes
            criterion = np.percentile(familiarities, 100-p_old) # set criterion
            r_old = familiarities[0:36]
            f_old = familiarities[36:72]
            r_new = familiarities[72:90]
            f_new = familiarities[90:108]
            new_cat = familiarities[108:144]
            r_old_hits = np.sum(r_old > criterion) / 36
            f_old_hits = np.sum(f_old > criterion) / 36
            r_new_hits = np.sum(r_new > criterion) / 18
            f_new_hits = np.sum(f_new > criterion) / 18
            new_cat_hits = np.sum(new_cat > criterion) / 36
            sim_list.append([r_old_hits, f_old_hits, r_new_hits, f_new_hits, new_cat_hits])
        means = np.mean(sim_list, axis=0)
        r_sq_value = r_sq(means, emp_data)
        means = means.tolist()
        means = [round(i, 2) for i in means]
        sds = np.std(sim_list, axis=0, ddof=1)
        sds = sds.tolist()
        sds = [round(i, 2) for i in sds]
        print('\n')
        print([l, round(F, 2), 'means:', means, round(r_sq_value, 4)])
        print([l, round(F, 2), 'sds:', sds, 'N/A'])

['L', 'F', 'mean/sd', ['R old', 'F old', 'R rel', 'F rel', 'Unrel'], 'R-squared']


[0.01, 0.67, 'means:', [0.78, 0.57, 0.58, 0.43, 0.14], 0.9824]
[0.01, 0.67, 'sds:', [0.08, 0.09, 0.11, 0.11, 0.07], 'N/A']


[0.03, 0.67, 'means:', [0.95, 0.64, 0.57, 0.25, 0.01], 0.9149]
[0.03, 0.67, 'sds:', [0.04, 0.07, 0.12, 0.09, 0.01], 'N/A']


[0.05, 0.67, 'means:', [0.98, 0.66, 0.54, 0.17, 0.0], 0.8509]
[0.05, 0.67, 'sds:', [0.02, 0.06, 0.12, 0.08, 0.0], 'N/A']


[0.1, 0.67, 'means:', [1.0, 0.7, 0.52, 0.08, 0.0], 0.7835]
[0.1, 0.67, 'sds:', [0.01, 0.06, 0.13, 0.07, 0.0], 'N/A']


[0.01, 0.8, 'means:', [0.75, 0.63, 0.54, 0.46, 0.12], 0.9217]
[0.01, 0.8, 'sds:', [0.08, 0.09, 0.12, 0.12, 0.06], 'N/A']


[0.03, 0.8, 'means:', [0.91, 0.73, 0.46, 0.27, 0.0], 0.818]
[0.03, 0.8, 'sds:', [0.05, 0.06, 0.12, 0.1, 0.01], 'N/A']


[0.05, 0.8, 'means:', [0.95, 0.75, 0.4, 0.18, 0.0], 0.7355]
[0.05, 0.8, 'sds:', [0.04, 0.06, 0.12, 0.09, 0.0], 'N/A']


[0.1, 0.8, 'means:', [0.98, 0.8, 0.32, 0.1, 0.0], 0.6273]
[0.

In [72]:
# Run Experiment 1 simulation with OrBEAGLE representations
emp_data = np.array([0.72619048, 0.52947846, 0.61111111, 0.42630385, 0.16326531]) # empirical data for computing r-squared
ls = [.1, .15, .2, .3] # learning rate
fs = [2/3, 4/5] # directed forgetting parameter
t = 3 # tau parameter
p_old = 50 # percent of items to be classifed "OLD"
print(['L', 'F', 'mean/sd', ['R old', 'F old', 'R rel', 'F rel', 'Unrel'], 'R-squared'])
for F in fs:
    for l in ls:
        f = l * F
        sim_list = []
        for s in range(0, 800):
            if s <= 200:
                stims = get_stim_list(test_a)
            elif s <= 400:
                stims = get_stim_list(test_b)
            elif s <= 600:
                stims = get_stim_list(test_c)
            else:
                stims = get_stim_list(test_d)
            memory = make_matrix(stims[0:72], seriol_matrix) # get vectors for encoded items
            memory[0:36] = memory[0:36] * np.random.choice([0, 1], size=(36, len(memory[0])), p=[1-l, l]) # randomly encode elements with probability L
            memory[36:72] = memory[36:72] * np.random.choice([0, 1], size=(36, len(memory[0])), p=[1-f, f]) # randomly encode elements with probability L*F
            probes = make_matrix(stims, seriol_matrix) # get vectors for recognition probes
            familiarities = echo_intensity(probes, memory, tau=t) # retrival: compute familairity for all probes
            criterion = np.percentile(familiarities, 100-p_old) # set criterion
            r_old = familiarities[0:36]
            f_old = familiarities[36:72]
            r_new = familiarities[72:90]
            f_new = familiarities[90:108]
            new_cat = familiarities[108:144]
            r_old_hits = np.sum(r_old > criterion) / 36
            f_old_hits = np.sum(f_old > criterion) / 36
            r_new_hits = np.sum(r_new > criterion) / 18
            f_new_hits = np.sum(f_new > criterion) / 18
            new_cat_hits = np.sum(new_cat > criterion) / 36
            sim_list.append([r_old_hits, f_old_hits, r_new_hits, f_new_hits, new_cat_hits])
        means = np.mean(sim_list, axis=0)
        r_sq_value = r_sq(means, emp_data)
        means = means.tolist()
        means = [round(i, 2) for i in means]
        sds = np.std(sim_list, axis=0, ddof=1)
        sds = sds.tolist()
        sds = [round(i, 2) for i in sds]
        print('\n')
        print([l, round(F, 2), 'means:', means, round(r_sq_value, 4)])
        print([l, round(F, 2), 'sds:', sds, 'N/A'])

['L', 'F', 'mean/sd', ['R old', 'F old', 'R rel', 'F rel', 'Unrel'], 'R-squared']


[0.1, 0.67, 'means:', [0.78, 0.54, 0.63, 0.44, 0.15], 0.9987]
[0.1, 0.67, 'sds:', [0.11, 0.11, 0.14, 0.12, 0.08], 'N/A']


[0.15, 0.67, 'means:', [0.85, 0.54, 0.67, 0.4, 0.08], 0.9949]
[0.15, 0.67, 'sds:', [0.09, 0.09, 0.13, 0.12, 0.06], 'N/A']


[0.2, 0.67, 'means:', [0.89, 0.53, 0.69, 0.37, 0.05], 0.9879]
[0.2, 0.67, 'sds:', [0.08, 0.09, 0.13, 0.11, 0.04], 'N/A']


[0.3, 0.67, 'means:', [0.94, 0.53, 0.72, 0.31, 0.01], 0.9737]
[0.3, 0.67, 'sds:', [0.05, 0.07, 0.12, 0.1, 0.02], 'N/A']


[0.1, 0.8, 'means:', [0.74, 0.6, 0.59, 0.47, 0.13], 0.9664]
[0.1, 0.8, 'sds:', [0.11, 0.11, 0.14, 0.13, 0.07], 'N/A']


[0.15, 0.8, 'means:', [0.8, 0.61, 0.6, 0.44, 0.07], 0.9758]
[0.15, 0.8, 'sds:', [0.1, 0.1, 0.13, 0.12, 0.06], 'N/A']


[0.2, 0.8, 'means:', [0.83, 0.62, 0.61, 0.41, 0.04], 0.9798]
[0.2, 0.8, 'sds:', [0.09, 0.1, 0.13, 0.12, 0.04], 'N/A']


[0.3, 0.8, 'means:', [0.89, 0.62, 0.61, 0.37, 0.01], 0.9738]
[0.3

In [73]:
# Run Experiment 2 simulation with OrBEAGLE representations
emp_data = np.array([0.68026005, 0.53546099, 0.53073286, 0.41371158, 0.21394799]) # empirical data for computing r-squared
ls = [.01, .03, .05, .1] # learning rate
fs = [2/3, 4/5] # directed forgetting parameter
t = 3 # tau parameter
p_old = 50 # percent of items to be classifed "OLD"
print(['L', 'F', 'mean/sd', ['R old', 'F old', 'R rel', 'F rel', 'Unrel'], 'R-squared'])
for F in fs:
    for l in ls:
        f = l * F
        sim_list = []
        for s in range(0, 800):
            if s <= 200:
                stims = get_stim_list(test_a_2)
            elif s <= 400:
                stims = get_stim_list(test_b_2)
            elif s <= 600:
                stims = get_stim_list(test_c_2)
            else:
                stims = get_stim_list(test_d_2)
            memory = make_matrix(stims[0:72], orbeagle_matrix) # get vectors for encoded items
            memory[0:36] = memory[0:36] * np.random.choice([0, 1], size=(36, len(memory[0])), p=[1-l, l]) # randomly encode elements with probability L
            memory[36:72] = memory[36:72] * np.random.choice([0, 1], size=(36, len(memory[0])), p=[1-f, f]) # randomly encode elements with probability L*F
            probes = make_matrix(stims, orbeagle_matrix) # get vectors for recognition probes
            familiarities = echo_intensity(probes, memory, tau=t) # retrival: compute familairity for all probes
            criterion = np.percentile(familiarities, 100-p_old) # set criterion
            r_old = familiarities[0:36]
            f_old = familiarities[36:72]
            r_new = familiarities[72:90]
            f_new = familiarities[90:108]
            new_cat = familiarities[108:144]
            r_old_hits = np.sum(r_old > criterion) / 36
            f_old_hits = np.sum(f_old > criterion) / 36
            r_new_hits = np.sum(r_new > criterion) / 18
            f_new_hits = np.sum(f_new > criterion) / 18
            new_cat_hits = np.sum(new_cat > criterion) / 36
            sim_list.append([r_old_hits, f_old_hits, r_new_hits, f_new_hits, new_cat_hits])
        means = np.mean(sim_list, axis=0)
        r_sq_value = r_sq(means, emp_data)
        means = means.tolist()
        means = [round(i, 2) for i in means]
        sds = np.std(sim_list, axis=0, ddof=1)
        sds = sds.tolist()
        sds = [round(i, 2) for i in sds]
        print('\n')
        print([l, round(F, 2), 'means:', means, round(r_sq_value, 4)])
        print([l, round(F, 2), 'sds:', sds, 'N/A'])

['L', 'F', 'mean/sd', ['R old', 'F old', 'R rel', 'F rel', 'Unrel'], 'R-squared']


[0.01, 0.67, 'means:', [0.76, 0.57, 0.57, 0.42, 0.18], 0.9992]
[0.01, 0.67, 'sds:', [0.08, 0.09, 0.12, 0.11, 0.08], 'N/A']


[0.03, 0.67, 'means:', [0.95, 0.63, 0.58, 0.25, 0.01], 0.9677]
[0.03, 0.67, 'sds:', [0.04, 0.07, 0.12, 0.09, 0.01], 'N/A']


[0.05, 0.67, 'means:', [0.98, 0.65, 0.56, 0.17, 0.0], 0.9264]
[0.05, 0.67, 'sds:', [0.02, 0.06, 0.12, 0.08, 0.0], 'N/A']


[0.1, 0.67, 'means:', [1.0, 0.68, 0.55, 0.09, 0.0], 0.8777]
[0.1, 0.67, 'sds:', [0.01, 0.06, 0.12, 0.07, 0.0], 'N/A']


[0.01, 0.8, 'means:', [0.74, 0.61, 0.53, 0.45, 0.16], 0.9786]
[0.01, 0.8, 'sds:', [0.08, 0.08, 0.12, 0.11, 0.07], 'N/A']


[0.03, 0.8, 'means:', [0.91, 0.71, 0.48, 0.28, 0.0], 0.9341]
[0.03, 0.8, 'sds:', [0.05, 0.07, 0.12, 0.1, 0.01], 'N/A']


[0.05, 0.8, 'means:', [0.95, 0.74, 0.42, 0.19, 0.0], 0.8685]
[0.05, 0.8, 'sds:', [0.04, 0.06, 0.11, 0.09, 0.0], 'N/A']


[0.1, 0.8, 'means:', [0.98, 0.79, 0.34, 0.1, 0.0], 0.7801]

In [74]:
# Run Experiment 2 simulation with SERIOL2 representations
emp_data = np.array([0.68026005, 0.53546099, 0.53073286, 0.41371158, 0.21394799]) # empirical data for computing r-squared
ls = [.1, .15, .2, .3] # learning rate
fs = [2/3, 4/5] # directed forgetting parameter
t = 3 # tau parameter
p_old = 50 # percent of items to be classifed "OLD"
print(['L', 'F', 'mean/sd', ['R old', 'F old', 'R rel', 'F rel', 'Unrel'], 'R-squared'])
for F in fs:
    for l in ls:
        f = l * F
        sim_list = []
        for s in range(0, 800):
            if s <= 200:
                stims = get_stim_list(test_a_2)
            elif s <= 400:
                stims = get_stim_list(test_b_2)
            elif s <= 600:
                stims = get_stim_list(test_c_2)
            else:
                stims = get_stim_list(test_d_2)
            memory = make_matrix(stims[0:72], seriol_matrix) # get vectors for encoded items
            memory[0:36] = memory[0:36] * np.random.choice([0, 1], size=(36, len(memory[0])), p=[1-l, l]) # randomly encode elements with probability L
            memory[36:72] = memory[36:72] * np.random.choice([0, 1], size=(36, len(memory[0])), p=[1-f, f]) # randomly encode elements with probability L*F
            probes = make_matrix(stims, seriol_matrix) # get vectors for recognition probes
            familiarities = echo_intensity(probes, memory, tau=t) # retrival: compute familairity for all probes
            criterion = np.percentile(familiarities, 100-p_old) # set criterion
            r_old = familiarities[0:36]
            f_old = familiarities[36:72]
            r_new = familiarities[72:90]
            f_new = familiarities[90:108]
            new_cat = familiarities[108:144]
            r_old_hits = np.sum(r_old > criterion) / 36
            f_old_hits = np.sum(f_old > criterion) / 36
            r_new_hits = np.sum(r_new > criterion) / 18
            f_new_hits = np.sum(f_new > criterion) / 18
            new_cat_hits = np.sum(new_cat > criterion) / 36
            sim_list.append([r_old_hits, f_old_hits, r_new_hits, f_new_hits, new_cat_hits])
        means = np.mean(sim_list, axis=0)
        r_sq_value = r_sq(means, emp_data)
        means = means.tolist()
        means = [round(i, 2) for i in means]
        sds = np.std(sim_list, axis=0, ddof=1)
        sds = sds.tolist()
        sds = [round(i, 2) for i in sds]
        print('\n')
        print([l, round(F, 2), 'means:', means, round(r_sq_value, 4)])
        print([l, round(F, 2), 'sds:', sds, 'N/A'])

['L', 'F', 'mean/sd', ['R old', 'F old', 'R rel', 'F rel', 'Unrel'], 'R-squared']


[0.1, 0.67, 'means:', [0.77, 0.52, 0.64, 0.44, 0.17], 0.9577]
[0.1, 0.67, 'sds:', [0.09, 0.1, 0.13, 0.12, 0.09], 'N/A']


[0.15, 0.67, 'means:', [0.85, 0.52, 0.68, 0.41, 0.09], 0.9571]
[0.15, 0.67, 'sds:', [0.09, 0.1, 0.13, 0.12, 0.06], 'N/A']


[0.2, 0.67, 'means:', [0.89, 0.52, 0.7, 0.38, 0.05], 0.956]
[0.2, 0.67, 'sds:', [0.07, 0.09, 0.14, 0.11, 0.04], 'N/A']


[0.3, 0.67, 'means:', [0.95, 0.52, 0.74, 0.32, 0.01], 0.94]
[0.3, 0.67, 'sds:', [0.05, 0.07, 0.12, 0.1, 0.01], 'N/A']


[0.1, 0.8, 'means:', [0.73, 0.58, 0.6, 0.48, 0.16], 0.976]
[0.1, 0.8, 'sds:', [0.09, 0.1, 0.13, 0.13, 0.09], 'N/A']


[0.15, 0.8, 'means:', [0.79, 0.6, 0.61, 0.46, 0.07], 0.9855]
[0.15, 0.8, 'sds:', [0.09, 0.1, 0.13, 0.13, 0.06], 'N/A']


[0.2, 0.8, 'means:', [0.84, 0.61, 0.62, 0.42, 0.03], 0.9944]
[0.2, 0.8, 'sds:', [0.08, 0.09, 0.14, 0.12, 0.04], 'N/A']


[0.3, 0.8, 'means:', [0.89, 0.61, 0.63, 0.36, 0.0], 0.9984]
[0.3, 0.8